In [ ]:
# Install the new library
!pip install pycaret

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Build healthy/broken dataset

In [ ]:
path = '/kaggle'
input_path = path + '/input/gearbox-fault-diagnosis-elaborated-datasets/gearbox-fault-diagnosis-elaborated-datasets/stdev/'
broken_dataset  = "broken30hz_stdev_100.csv"
healthy_dataset = "healthy30hz_stdev_100.csv"

In [ ]:
healthyDataset = pd.read_csv(input_path + healthy_dataset)
brokenDataset = pd.read_csv(input_path + broken_dataset)

dataset = pd.concat([healthyDataset, brokenDataset], axis=0)
dataset.describe()

In order to demonstrate the predict_model() function on unseen data, a small sample of records has been withheld from the original dataset to be used for predictions. This should not be confused with a train/test split as this particular split is performed to simulate a real life scenario. Another way to think about this is that these 1200 records are not available at the time when the machine learning experiment was performed.

In [ ]:
data = dataset.sample(frac=0.95, random_state=786)
data_unseen = dataset.drop(data.index)
data.reset_index(inplace=True, drop=True)
data_unseen.reset_index(inplace=True, drop=True)
print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

# Setting up Environment in PyCaret

In [ ]:
from pycaret.classification import *

In [ ]:
exp_clf101 = setup(data = data, target = 'failure', session_id=123) 

Once the setup has been succesfully executed it prints the information grid which contains several important pieces of information. Most of the information is related to the pre-processing pipeline which is constructed when `setup()` is executed. Be aware of a few important things to note at this stage include:

- **session_id :**  A pseduo-random number distributed as a seed in all functions for later reproducibility. If no `session_id` is passed, a random number is automatically generated that is distributed to all functions. In this experiment, the `session_id` is set as `123` for later reproducibility.<br/>
<br/>
- **Target Type :**  Binary or Multiclass. The Target type is automatically detected and shown. There is no difference in how the experiment is performed for Binary or Multiclass problems. All functionalities are identical.<br/>
<br/>
- **Label Encoded :**  When the Target variable is of type string (i.e. 'Yes' or 'No') instead of 1 or 0, it automatically encodes the label into 1 and 0 and displays the mapping (0 : No, 1 : Yes) for reference. In this experiment no label encoding is required since the target variable is of type numeric. <br/>
<br/>
- **Original Data :**  Displays the original shape of the dataset. <br/>
<br/>
- **Missing Values :**  When there are missing values in the original data this will show as True. For this experiment there are no missing values in the dataset. <br/>
<br/>
- **Numeric Features :**  The number of features inferred as numeric. <br/>
<br/>
- **Categorical Features :**  The number of features inferred as categorical. <br/>
<br/>
- **Transformed Train Set :**  Displays the shape of the transformed training set. <br/>
<br/>
- **Transformed Test Set :**  Displays the shape of the transformed test/hold-out set. <br/>

Notice how a few tasks that are imperative to perform modeling are automatically handled such as missing value imputation (in this case there are no missing values in the training data, but we still need imputers for unseen data), categorical encoding etc. Most of the parameters in `setup()` are optional and used for customizing the pre-processing pipeline.

# Comparing All Models

Comparing all models to evaluate performance is the recommended starting point for modeling once the setup is completed (unless you exactly know what kind of model you need, which is often not the case). This function trains all models in the model library and scores them using stratified cross validation for metric evaluation. The output prints a score grid that shows average Accuracy, AUC, Recall, Precision, F1, Kappa, and MCC accross the folds (10 by default) along with training times.

In [ ]:
best_model = compare_models()

In [ ]:
print(best_model)

# Create a Model: Logistic Regression

`create_model` is the most granular function in PyCaret and is often the foundation behind most of the PyCaret functionalities. As the name suggests this function trains and evaluates a model using cross validation that can be set with `fold` parameter. The output prints a score grid that shows Accuracy, AUC, Recall, Precision, F1, Kappa and MCC by fold. 

We will create the Logistic Regression model ('lr).

There are 18 classifiers available in the model library of PyCaret. To see list of all classifiers either check the `docstring` or use `models` function to see the library.

In [ ]:
models

In [ ]:
lr = create_model('lr')

## Tune the model & evaluate

In [ ]:
tuned_lr = tune_model(lr)

In [ ]:
evaluate_model(tuned_lr)

## Plot the model

In [ ]:
plot_model(tuned_lr, plot = 'auc')

In [ ]:
plot_model(tuned_lr, plot = 'pr')

In [ ]:
plot_model(tuned_lr, plot='feature')

In [ ]:
plot_model(lr, plot = 'confusion_matrix')

In [ ]:
plot_model(tuned_lr, 'class_report')

## Predict on test set

In [ ]:
predict_model(tuned_lr)